# IR3 Last column (counts) of the patients for the BL visit.

In [1]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
import os
import glob
import functools

In [3]:
#!pip install dask[complete];
#!pip install --upgrade pandas "dask[complete]"
#python -m pip install "dask[dataframe]" --upgrade

In [13]:
path_to_files="/scratch/znazari/PPMI_ver_sep2022/RNA_Seq_data/star_ir3/counts/"

Note that the counts file in the IR3 is around 152 G

In [14]:
#reading the files which are in BL (Base line) visit.
specific_word = 'BL'
ending_pattern = '*.txt'
file_pattern = f'*{specific_word}*.{ending_pattern}'
file_paths = glob.glob(path_to_files + file_pattern)

In [15]:
filename = 'bl.txt'
file_path_2 = os.path.join(path_to_files, filename)
bl_files = pd.read_csv(file_path_2,header=None)
bl_files

0
0     PPMI-Phase1-IR3.10874.BL.PP0047-6931.5104-SL-0...
1     PPMI-Phase1-IR3.12499.BL.PP0042-3049.5104-SL-1...
2     PPMI-Phase1-IR3.12593.BL.0003149216.5104-SL-13...
3     PPMI-Phase1-IR3.13039.BL.PP0048-8378.5104-SL-1...
4     PPMI-Phase1-IR3.13424.BL.0003180024.5104-SL-18...
...                                                 ...
1525  PPMI-Phase2-IR3.65003.BL.0003195560.5104-SL-03...
1526  PPMI-Phase2-IR3.65005.BL.0003451950.5104-SL-26...
1527  PPMI-Phase2-IR3.65008.BL.PP0041-9827.5104-SL-2...
1528  PPMI-Phase2-IR3.70239.BL.0003414527.5104-SL-39...
1529  PPMI-Phase2-IR3.85242.BL.0000329301.5104-SL-24...

[1530 rows x 1 columns]

In [56]:
# We define a function where we can take the second phrase seperated by dot. The second phrase 
# is the patient ID. So with this functin we want to get the patient IDs from their file's name
def function_names(fname):
    tokens=fname.split('.')
    return tokens[1]

In [60]:
bl_list=[]
for i in range(len(bl_files)):
    bl_list=bl_list+[function_names(bl_files.iloc[i][0])]
    
bl_list;

In [61]:
len(bl_list)

1530

In [ ]:
# here we read all the files with with base visit from the count folder where we have all the files
# for all the patients and all the visit.
# here we have a list of all files with base visit. 
list_bl_files=[]
for i in range(len(bl_files)):
    list_bl_files=list_bl_files+[pd.read_csv(path1/bl_files.iloc[i][0],skiprows=1,delimiter='\t', index_col=[0])]

In [16]:
%%time
dask_files = dd.read_csv(file_paths,skiprows=1,delimiter='\t')

CPU times: user 277 ms, sys: 108 ms, total: 384 ms
Wall time: 381 ms


In [59]:
dask_files.npartitions

1530

In [32]:
second = dask_files.get_partition(1)

In [54]:
oh3 = second.columns.values[6]

In [58]:
oh3

'/home/dtg-scratch-01/Projects/PPMI/working/starB38/symLinks/phase1/PPMI-Phase1-IR1.3385.BL.0003288237.5104-SL-0092.Aligned.out.sorted.bam'

In [57]:
 function_names(oh3)

'3385'

In [20]:
dask_files

Dask DataFrame Structure:
                  Geneid     Chr   Start     End  Strand Length /home/dtg-scratch-01/Projects/PPMI/working/starB38/symLinks/phase1/PPMI-Phase1-IR1.3385.BL.0003288237.5104-SL-0092.Aligned.out.sorted.bam
npartitions=1530                                                                                                                                                                                         
                  object  object  object  object  object  int64                                                                                                                                     int64
                     ...     ...     ...     ...     ...    ...                                                                                                                                       ...
...                  ...     ...     ...     ...     ...    ...                                                                                                                                       ...
                     ...     ...     ...     ...     ...    ...                                                                                                                                       ...
                     ...     ...     ...     ...     ...    ...                                                                                                                                       ...
Dask Name: read-csv, 1 graph layer

In [7]:
dask_files.get_partition(2)

Dask DataFrame Structure:
               Geneid     Chr   Start     End  Strand Length /home/dtg-scratch-01/Projects/PPMI/working/starB38/symLinks/phase1/PPMI-Phase1-IR1.3385.BL.0003288237.5104-SL-0092.Aligned.out.sorted.bam
npartitions=1                                                                                                                                                                                         
               object  object  object  object  object  int64                                                                                                                                     int64
                  ...     ...     ...     ...     ...    ...                                                                                                                                       ...
Dask Name: get-partition, 2 graph layers

In [15]:
type(dask_files)

dask.dataframe.core.DataFrame

In [22]:
last_columns = dask_files.iloc[:, -1:]
type(last_columns)
last_columns

Dask DataFrame Structure:
                 /home/dtg-scratch-01/Projects/PPMI/working/starB38/symLinks/phase1/PPMI-Phase1-IR1.3385.BL.0003288237.5104-SL-0092.Aligned.out.sorted.bam
npartitions=1530                                                                                                                                          
                                                                                                                                                     int64
                                                                                                                                                       ...
...                                                                                                                                                    ...
                                                                                                                                                       ...
                                                                                                                                                       ...
Dask Name: getitem, 2 graph layers

In [ ]:
%%time
dfSave = pd.DataFrame()
for i in range(len(bl_files)):
    print(i)
    ddf = dd.read_csv(path1/bl_files.iloc[i][0],skiprows=1,delimiter='\t')
    
    dfSave[bl_list[i]] = df.iloc[:,-1:]
    
    
    del df